<a href="https://colab.research.google.com/github/lenyabloko/SemEval2020/blob/master/SemEval2020_Task5_Subtask_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/train.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/test.csv /content

In [0]:
!pip install allennlp
!pip install nltk

from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/elmo-constituency-parser-2018.03.14.tar.gz")

In [0]:
import nltk
nltk.download('punkt')

import pandas as pd

from progress.bar import IncrementalBar
from time import sleep

prefix = '/content/'
train_df = pd.read_csv(prefix + 'train.csv', header=None)
train_df=train_df.drop(index=0)
  
train_df = pd.DataFrame({
    'id':train_df[0].replace(r"[A-Za-z]", "", regex=True),
    'labels':train_df[1],
    'alpha':['a']*train_df.shape[0],
    'text': train_df[2].replace(r'\n', ' ', regex=True)
})

bar = IncrementalBar('Simplifying sentences', max = train_df.count)
for ind in train_df.index: 
  prediction = predictor.predict(sentence=train_df['text'][ind])
  train_df['text'][ind] = simplify()
  bar.next()
  sleep(1)

bar.finish()
train_df.to_csv(prefix+'train.tsv', sep='\t', index=False, header=False)
train_df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def extract_modal(obj):
    """Pull all values of specified key from nested JSON."""
    arr = list()

    def extract(obj, arr):
        
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr)
                    
                elif k == 'nodeType' and v == 'MD':
                    arr.append(v)
                elif k == 'nodeType' and v == 'NP' and not 'MD' in arr:
                    arr.append(v)    

        elif isinstance(obj, list):
            for item in obj:
              if 'MD' in arr:
                  arr.append(item) 
              else:    
                if len(arr) > 0 and arr[len(arr)-1] == 'NP':
                    arr[len(arr)-1] = item
                extract(item, arr)
                
        return arr

    results = extract(obj, arr)
    return results

In [0]:
def separate_children(obj):
    """Pull all values of specified key from nested JSON."""
    arr = list()

    def separate(obj, arr):
        
        if isinstance(obj, dict):

            for k, v in obj.items():
              if k != 'children':
                if isinstance(v, (dict, list)):
                    separate(v, arr)
                elif k == 'word':      
                  arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                separate(item, arr)
         
        return arr

    results = separate(obj, arr)
    return results

In [0]:
def extract_constituents():
 for key, value in prediction.items():
  if (key=='hierplane_tree'):
    for key, value in value.items():
      if (key=='root'):
        for key, value in value.items():
          if (key == 'children'):
            return extract_modal(value)

In [0]:
import nltk
from nltk.tokenize import word_tokenize

def simplify():
  simpler = ''
  constit = list(extract_constituents())

  con = list()
  for c in reversed(constit):
      p = separate_children(c)
      for parent in list(p):
        tokens = word_tokenize(parent)
        if len(tokens) > 1:
          con.append(parent)

  if (len(con) > 1):      
    return con[1]+' '+con[0]
  if (len(con) > 0):      
    return con[0]
  return ''

In [104]:
prediction = predictor.predict(
  sentence="The new request, if approved, would keep the military forces on the border through Jan."
)
simplify()

'keep the military forces on the border through Jan.'